# AIS Examples

This example uses AIS data to show how to read and write information using PyMEOS.
It is divided in 3 sections, each corresponding to one MEOS example:
- [Reading from File](https://libmeos.org/tutorialprograms/meos_read_ais/)
- [Assembling Trips](https://libmeos.org/tutorialprograms/meos_assemble_ais/)
- [Storing in MobilityDB](https://libmeos.org/tutorialprograms/meos_store_ais/)

This example uses the plotting and database capabilities of PyMEOS. To get the necessary dependencies, you can install pymeos specifying this options:
```shell
pip install pymeos[db,plot]
```

In [1]:
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
from pymeos import *

pymeos_initialize()

ModuleNotFoundError: No module named 'shapely'

## Reading from File ([MEOS Example](https://libmeos.org/tutorialprograms/meos_read_ais/))
In this section, AIS data is read from a CSV file. We then use the read information to recreate the trajectories of the ships.



First, let's read the CSV file using pandas.

In [ ]:
ais = pd.read_csv('./data/aisinput.csv')
ais.head()

Now, we will create the PyMEOS object representing the position and the SOG.

In [ ]:
ais['point'] = ais.apply(lambda row: TGeogPointInst(point=(row['longitude'], row['latitude']), timestamp=row['t']),
                         axis=1)
ais['sog'] = ais.apply(lambda row: TFloatInst(value=row['sog'], timestamp=row['t']), axis=1)
ais['geometry'] = ais.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
ais.drop(['latitude', 'longitude'], axis=1, inplace=True)
ais.head()

## Assembling Trips ([MEOS Example](https://libmeos.org/tutorialprograms/meos_assemble_ais/))

Now, we will create the trajectory (TGeogPointSeq) and the SOG evolution (TFloatSeq) for every ship (identified by the mmsi) using the instants we have created.

In [ ]:
trajectories = ais.groupby('mmsi').aggregate(
    {
        'point': TGeogPointSeq.from_instants,
        'sog': TFloatSeq.from_instants
    }
).rename({'point': 'trajectory'}, axis=1)
trajectories['distance'] = trajectories['trajectory'].apply(lambda t: t.length())
trajectories.head()

Here we can see that PyMEOS has been able to greatly reduce the number of points stored (and thus memory used) without losing any information.

In [ ]:
pd.concat([ais.groupby('mmsi')['t'].count().rename('original #points'),
           trajectories['trajectory'].apply(lambda t: t.num_instants).rename('PyMEOS #points')],
          axis=1)

We can visualize the trajectories and the SOG evolutions by plotting them.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
for _, ship in trajectories.iterrows():
    ship['trajectory'].plot(axes=axes[0])
    ship['sog'].plot(axes=axes[1])
plt.show()

## Calculating temporal distances

Let's find all AIS records within a given temporal distance of the green trajectory (mmsi=257136000)

In [ ]:
MMSI = 257136000
trajectories[trajectories.index.isin([MMSI])]

In [ ]:
def distance_to_traj(row):
    tpt = row['point']
    traj = trajectories[trajectories.index.isin([MMSI])].trajectory.iloc[0]
    d = traj.distance(tpt)
    if d and row.mmsi != MMSI:
        return d.value()
    return None

ais['dist'] = ais.apply(distance_to_traj, axis=1)
ais

In [ ]:
ais[(ais.dist>0) & (ais.dist<20000)]

In [ ]:
import geopandas as gpd

In [ ]:
MAX_DIST = 20000
dwithin = gpd.GeoDataFrame(ais[(ais.dist>0) & (ais.dist<=MAX_DIST)][['mmsi','geometry','dist']], crs=4326)
allais = gpd.GeoDataFrame(ais[ais.mmsi!=MMSI][['mmsi','geometry','dist']], crs=4326)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))    # define fig and axes
trajectories[trajectories.index.isin([257136000])].iloc[0]['trajectory'].plot(axes=ax)
allais.plot(ax=ax, color='gray')
dwithin.plot(ax=ax, color='red')

# Storing in MobilityDB ([MEOS Example](https://libmeos.org/tutorialprograms/meos_store_ais/))

Now we will store the temporal objects we have created in MobilityDB.
To connect to MobilityDB (PostgreSQL), `psycopg2` is used. You can install it yourself or install `pymeos` with the `dbp2` option:
```shell
pip install pymeos[dbp2]
```
If you don't have MobilityDB set up, you can use the Docker image to quickly start:
```shell
docker run -d -p 5432:5432 --name pymeos-demo-db mobilitydb/mobilitydb:14-3.2-develop
```
MobilityDB also supports `asyncpg` and `psycopg` (psycopg3)

First, set up the connection parameters. Change any of the following values according to your configuration

In [ ]:
from pymeos.db.psycopg2 import MobilityDB

host = 'localhost'
port = 5432
db = 'mobilitydb'
user = 'docker'
password = 'docker'

connection = MobilityDB.connect(host=host, port=port, database=db, user=user, password=password)
cursor = connection.cursor()

Now, we will create the table where we will write our objects.

In [ ]:
cursor.execute("DROP TABLE IF EXISTS public.PyMEOS_demo;")
cursor.execute("CREATE TABLE public.PyMEOS_demo"
               "(MMSI integer, trajectory public.tgeogpoint, SOG public.tfloat);")
connection.commit()

Let's insert now the rows of the DataFrame into the MobilityDB table

In [ ]:
for mmsi, row in trajectories.iterrows():
    cursor.execute(f"INSERT INTO public.PyMEOS_demo(MMSI, trajectory, SOG) "
                   f"VALUES ({mmsi}, '{row.trajectory}', '{row.sog}');")
connection.commit()

Now, we will read one of the records that we just wrote

In [ ]:
cursor.execute("SELECT * FROM public.PyMEOS_demo WHERE MMSI = 257136000;")
mmsi, trajectory, sog = cursor.fetchone()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
trajectory.plot(axes=axes[0])
sog.plot(axes=axes[1])
plt.suptitle(f'Ship {mmsi}')
plt.show()

Finally, let's close the connection and wrap everything up

In [ ]:
connection.commit()
cursor.close()
pymeos_finalize()